# Structured Outputs（结构化输出）基础教程

## 学习目标 ⭐
* 理解 Structured Output 的定义与优势
* 掌握在 Prompt 中引导 Claude 生成 JSON / YAML 等结构化数据的技巧
* 能够在资讯摘要与交易趋势场景中设计高质量结构化输出
* 熟练使用 Claude API（流式模式）获取结构化结果并进行后续处理

> **预计耗时**：约 15 分钟

---

## 前言（2 分钟）
在银行业务实践中，我们常需要将大模型输出直接交由后端系统解析并进入风控、合规或报表流程。与自然语言相比，**结构化输出（Structured Output）**更易于机器读取和后续自动化处理，因此成为 Prompt 工程的重要补充模块。


## 1. 核心概念（4 分钟）

### 什么是结构化输出？
**结构化输出（Structured Output）**是指通过精心设计的 Prompt，让大模型直接输出符合 JSON / YAML / CSV 等格式规范的文本，避免后续正则解析。

### 主要优势
1. **易解析、少歧义** → 提升自动化程度
2. **方便风控校验** → 提高合规性  
3. **与前端/后端系统快速集成** → 降低工程复杂度

### Claude 的结构化输出支持
- **Prefill（预填充）**：通过在 `assistant` 角色中预填充 `{` 来强制 JSON 格式输出
- **System 指令**：在系统提示中明确输出格式要求
- **一致性增强**：通过示例和约束提高输出稳定性

### TACOS 框架补充
在 **Output** 维度中明确指定格式与字段：
- `"请以合法 JSON 格式输出，不要添加多余文本"`
- `"输出必须包含以下字段：title, summary, sentiment, risk_level"`


In [1]:
# 环境配置与依赖安装
%pip install anthropic python-dotenv requests httpx --quiet

import anthropic
import json
import os
from dotenv import load_dotenv
import httpx
from typing import Dict, Any


Note: you may need to restart the kernel to use updated packages.


In [2]:
# API 配置 - 支持流式输出
load_dotenv()

client = anthropic.Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    http_client=httpx.Client(proxy="http://127.0.0.1:7890/")
)

def call_claude_structured(prompt: str, system: str = "", max_tokens: int = 1000, 
                          temperature: float = 0.3, prefill: str = "") -> str:
    """
    调用Claude API的结构化输出函数（流式输出）
    
    参数:
    - prompt: 用户输入的提示词
    - system: 系统提示词
    - max_tokens: 最大token数
    - temperature: 随机性参数（结构化输出建议使用较低值 0.1-0.3）
    - prefill: 预填充内容（用于强制格式）
    """
    messages = [{"role": "user", "content": prompt}]
    
    # 如果有预填充内容，添加 assistant 消息
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
    
    full_response = ""
    
    with client.messages.stream(
        model="claude-4-sonnet-20250514",
        max_tokens=max_tokens,
        temperature=temperature,
        system=system,
        messages=messages
    ) as stream:
        for text in stream.text_stream:
            print(text, end="", flush=True)
            full_response += text
    
    print()  # 添加换行
    return prefill + full_response if prefill else full_response

print("✅ Claude API 配置完成")


✅ Claude API 配置完成


## 2. 资讯摘要场景：结构化输出设计（3 分钟）

### 场景描述
银行需要对央行政策、财经新闻进行快速分析和风险评估，要求输出标准化的 JSON 格式便于后续系统处理。

### 设计要点
1. **明确字段结构**：标题、摘要、情感倾向、风险等级
2. **使用 Prefill 技术**：强制 JSON 格式输出
3. **增强一致性**：通过示例和约束提高稳定性


In [3]:
# 示例1：财经新闻结构化摘要
news_content = """
央行今日发布2024年第四季度货币政策执行报告。报告显示，人民币汇率保持基本稳定，
外汇储备规模继续保持在3.2万亿美元以上。房地产市场呈现企稳回升态势，
多个一线城市成交量环比上升15%。央行表示将继续实施稳健的货币政策，
保持流动性合理充裕，支持实体经济发展。同时强调将密切关注通胀走势，
适时调整政策工具组合。市场分析师认为，此次报告释放了积极信号，
预计短期内不会有大幅政策调整。
"""

# 系统提示词 - 强调输出格式
system_prompt = """你是专业的金融分析师，擅长将复杂的财经信息转化为结构化数据。
请严格按照JSON格式输出，不要添加任何额外的文本、解释或格式标记。
输出的JSON必须包含以下字段：title, summary, sentiment, risk_level, key_points。"""

# 用户提示词 - 明确任务和输出格式
user_prompt = f"""请分析以下财经新闻内容，并输出结构化的JSON格式分析结果：

新闻内容：
{news_content}

输出格式要求：
{{
  "title": "新闻标题总结",
  "summary": "核心内容摘要（不超过100字）",
  "sentiment": "positive/negative/neutral",
  "risk_level": "low/medium/high",
  "key_points": ["要点1", "要点2", "要点3"]
}}

请严格按照上述JSON格式输出，不要添加任何其他内容。"""

print("🔥 开始调用 Claude API 进行财经新闻结构化分析...\n")


🔥 开始调用 Claude API 进行财经新闻结构化分析...



In [4]:
# 使用 Prefill 技术强制 JSON 输出
result1 = call_claude_structured(
    prompt=user_prompt,
    system=system_prompt,
    temperature=0.2,  # 较低温度确保输出稳定
    prefill="{"  # 预填充强制以 JSON 开头
)

print("\n" + "="*50)
print("📊 解析结果验证：")
try:
    # 验证输出是否为合法 JSON
    parsed_result = json.loads(result1)
    print("✅ JSON 格式验证通过")
    print(f"📈 情感倾向: {parsed_result.get('sentiment', 'N/A')}")
    print(f"⚠️ 风险等级: {parsed_result.get('risk_level', 'N/A')}")
    print(f"📋 关键要点数: {len(parsed_result.get('key_points', []))}")
except json.JSONDecodeError as e:
    print(f"❌ JSON 解析失败: {e}")
    print("🔧 需要优化 Prompt 设计")



  "title": "央行发布四季度货币政策报告 房地产市场企稳回升",
  "summary": "央行发布2024年第四季度货币政策执行报告，显示人民币汇率基本稳定，外汇储备保持3.2万亿美元以上，房地产市场呈现企稳回升态势，一线城市成交量环比上升15%，将继续实施稳健货币政策。",
  "sentiment": "positive",
  "risk_level": "low",
  "key_points": ["人民币汇率保持基本稳定，外汇储备规模稳定在3.2万亿美元以上", "房地产市场企稳回升，一线城市成交量环比上升15%", "央行将继续实施稳健货币政策，保持流动性合理充裕", "市场预期短期内不会有大幅政策调整"]
}

📊 解析结果验证：
✅ JSON 格式验证通过
📈 情感倾向: positive
⚠️ 风险等级: low
📋 关键要点数: 4


## 3. 交易趋势场景：技术指标结构化分析（3 分钟）

### 场景描述
银行需要对股票价格走势进行技术分析，计算 RSI、MACD 等指标，并生成结构化的交易信号和风险评估。

### 核心挑战
1. **数值精度要求**：技术指标计算需要保持小数点精度
2. **多维度输出**：价格、指标、信号、风险需要分层结构化
3. **时间序列处理**：历史数据的结构化表示


In [5]:
# 示例2：股票技术指标结构化分析
stock_data = {
    "symbol": "000001.SZ",
    "name": "平安银行",
    "prices": [15.20, 15.35, 15.18, 15.42, 15.38, 15.55, 15.48, 15.62, 15.71, 15.68],
    "volumes": [12500000, 14200000, 11800000, 15600000, 13900000, 16800000, 15200000, 17500000, 18200000, 16900000],
    "dates": ["2024-01-10", "2024-01-11", "2024-01-12", "2024-01-15", "2024-01-16", "2024-01-17", "2024-01-18", "2024-01-19", "2024-01-22", "2024-01-23"]
}

# 系统提示词 - 技术分析专家
system_prompt_trading = """你是专业的量化交易分析师，精通股票技术分析和风险评估。
请基于提供的股票数据进行技术分析，并输出标准化的JSON格式结果。
计算结果请保持2位小数精度，交易信号请明确给出买入/卖出/观望建议。"""

# 用户提示词 - 技术指标分析
trading_prompt = f"""请分析以下股票数据，计算技术指标并生成交易信号：

股票信息：
- 代码：{stock_data['symbol']}
- 名称：{stock_data['name']}
- 价格序列：{stock_data['prices']}
- 成交量序列：{stock_data['volumes']}
- 日期序列：{stock_data['dates']}

输出格式要求（严格JSON）：
{{
  "basic_info": {{
    "symbol": "股票代码",
    "name": "股票名称",
    "current_price": 当前价格,
    "price_change": 涨跌幅百分比
  }},
  "technical_indicators": {{
    "rsi": RSI指标值,
    "ma5": 5日均线,
    "ma10": 10日均线,
    "volume_ratio": 量比
  }},
  "trading_signal": {{
    "action": "buy/sell/hold",
    "confidence": 信号强度(0-1),
    "target_price": 目标价位,
    "stop_loss": 止损价位
  }},
  "risk_assessment": {{
    "level": "low/medium/high",
    "factors": ["风险因素1", "风险因素2"],
    "recommendation": "投资建议"
  }}
}}

请严格按照JSON格式输出，保持数值精度。"""

print("📈 开始调用 Claude API 进行技术指标分析...\n")


📈 开始调用 Claude API 进行技术指标分析...



In [6]:
# 执行技术分析并验证结果
result2 = call_claude_structured(
    prompt=trading_prompt,
    system=system_prompt_trading,
    temperature=0.1,  # 极低温度确保数值计算一致性
    prefill="{"
)

print("\n" + "="*50)
print("📊 技术分析结果验证：")
try:
    # 解析交易分析结果
    trading_result = json.loads(result2)
    print("✅ JSON 格式验证通过")
    
    # 提取关键信息
    basic_info = trading_result.get('basic_info', {})
    technical = trading_result.get('technical_indicators', {})
    signal = trading_result.get('trading_signal', {})
    risk = trading_result.get('risk_assessment', {})
    
    print(f"🏦 股票: {basic_info.get('name', 'N/A')} ({basic_info.get('symbol', 'N/A')})")
    print(f"💰 当前价格: {basic_info.get('current_price', 'N/A')}")
    print(f"📊 RSI: {technical.get('rsi', 'N/A')}")
    print(f"🎯 交易信号: {signal.get('action', 'N/A')} (置信度: {signal.get('confidence', 'N/A')})")
    print(f"⚠️ 风险等级: {risk.get('level', 'N/A')}")
    
except json.JSONDecodeError as e:
    print(f"❌ JSON 解析失败: {e}")
    print("🔧 需要优化技术分析 Prompt")

print("\n⚠️ 重要提示：以上分析结果仅供参考，不构成投资建议。所有投资决策需经过银行风控系统严格审核。")



  "basic_info": {
    "symbol": "000001.SZ",
    "name": "平安银行",
    "current_price": 15.68,
    "price_change": 3.16
  },
  "technical_indicators": {
    "rsi": 68.42,
    "ma5": 15.48,
    "ma10": 15.42,
    "volume_ratio": 1.35
  },
  "trading_signal": {
    "action": "hold",
    "confidence": 0.65,
    "target_price": 16.20,
    "stop_loss": 15.20
  },
  "risk_assessment": {
    "level": "medium",
    "factors": ["RSI接近超买区域", "成交量放大但价格涨幅有限", "短期均线支撑较弱"],
    "recommendation": "建议观望，等待更明确的突破信号或回调买入机会"
  }
}

📊 技术分析结果验证：
✅ JSON 格式验证通过
🏦 股票: 平安银行 (000001.SZ)
💰 当前价格: 15.68
📊 RSI: 68.42
🎯 交易信号: hold (置信度: 0.65)
⚠️ 风险等级: medium

⚠️ 重要提示：以上分析结果仅供参考，不构成投资建议。所有投资决策需经过银行风控系统严格审核。


## 4. 高级技巧：一致性增强与错误处理（2 分钟）

### Prefill 技术要点
1. **强制格式开头**：使用 `{` 或 `[` 预填充
2. **减少歧义**：避免模型输出解释性文字
3. **提高成功率**：显著降低格式错误概率

### 一致性增强策略
1. **低温度参数**：结构化输出建议使用 0.1-0.3
2. **明确约束**：在 System 和 User prompt 中双重强调
3. **示例引导**：提供标准格式模板


In [7]:
# 高级技巧演示：错误处理与重试机制
def robust_structured_call(prompt: str, system: str = "", max_retries: int = 3) -> Dict[str, Any]:
    """
    带重试机制的结构化输出调用
    
    参数:
    - prompt: 用户提示词
    - system: 系统提示词
    - max_retries: 最大重试次数
    
    返回:
    - 解析成功的字典对象或错误信息
    """
    for attempt in range(max_retries):
        try:
            print(f"🔄 尝试第 {attempt + 1} 次调用...")
            
            # 调用 Claude API
            result = call_claude_structured(
                prompt=prompt,
                system=system,
                temperature=0.1 + attempt * 0.05,  # 逐渐提高温度
                prefill="{"
            )
            
            # 尝试解析 JSON
            parsed_result = json.loads(result)
            print(f"✅ 第 {attempt + 1} 次调用成功！")
            return {"success": True, "data": parsed_result, "attempts": attempt + 1}
            
        except json.JSONDecodeError as e:
            print(f"❌ 第 {attempt + 1} 次解析失败: {e}")
            if attempt == max_retries - 1:
                return {"success": False, "error": str(e), "attempts": max_retries}
            else:
                print("🔁 准备重试...")
                
        except Exception as e:
            print(f"💥 第 {attempt + 1} 次调用异常: {e}")
            if attempt == max_retries - 1:
                return {"success": False, "error": str(e), "attempts": max_retries}

# 测试重试机制
print("🧪 测试结构化输出重试机制：")
test_prompt = """请分析平安银行最新季报，输出JSON格式：
{
  "revenue": "营收（亿元）",
  "profit": "净利润（亿元）", 
  "growth_rate": "增长率（%）"
}"""

robust_result = robust_structured_call(test_prompt, system_prompt)
if robust_result["success"]:
    print(f"✅ 成功获取结构化结果，用时 {robust_result['attempts']} 次尝试")
else:
    print(f"❌ 重试 {robust_result['attempts']} 次后仍失败: {robust_result['error']}")


🧪 测试结构化输出重试机制：
🔄 尝试第 1 次调用...

  "title": "平安银行季报分析",
  "summary": "抱歉，我无法获取平安银行最新季报的实时数据。作为AI助手，我无法访问实时的财务数据或最新的季报信息。建议您查阅平安银行官方网站、交易所公告或专业财经平台获取最新季报数据。",
  "sentiment": "中性",
  "risk_level": "无法评估",
  "key_points": [
    "需要获取最新季报数据才能进行准确分析",
    "建议查阅官方渠道获取实时财务信息",
    "季报分析需要基于具体的财务数据"
  ]
}
✅ 第 1 次调用成功！
✅ 成功获取结构化结果，用时 1 次尝试


## 5. 常见错误与防御策略（2 分钟）

### 常见问题
1. **格式不一致**：有时输出额外的解释文字
2. **字段缺失**：部分必需字段未包含
3. **类型错误**：数值被输出为字符串

### 防御策略
1. **多重约束**：System + User + Prefill 三重保障
2. **验证机制**：解析后检查必需字段
3. **回退方案**：解析失败时的替代处理

### 银行业务应用建议
- 关键业务数据建议人工复核
- 建立完整的审计日志
- 设置合理的置信度阈值


In [8]:
# 字段验证示例
def validate_financial_analysis(data: Dict[str, Any]) -> Dict[str, Any]:
    """
    验证财经分析结果的完整性和合法性
    
    参数:
    - data: 待验证的JSON数据
    
    返回:
    - 验证结果和修正建议
    """
    validation_result = {
        "valid": True,
        "errors": [],
        "warnings": [],
        "suggestions": []
    }
    
    # 检查必需字段
    required_fields = ["title", "summary", "sentiment", "risk_level"]
    for field in required_fields:
        if field not in data:
            validation_result["valid"] = False
            validation_result["errors"].append(f"缺少必需字段: {field}")
    
    # 检查情感倾向值域
    if "sentiment" in data:
        valid_sentiments = ["positive", "negative", "neutral"]
        if data["sentiment"] not in valid_sentiments:
            validation_result["warnings"].append(f"情感倾向值异常: {data['sentiment']}")
            validation_result["suggestions"].append("建议使用: positive/negative/neutral")
    
    # 检查风险等级值域
    if "risk_level" in data:
        valid_risks = ["low", "medium", "high"]
        if data["risk_level"] not in valid_risks:
            validation_result["warnings"].append(f"风险等级值异常: {data['risk_level']}")
            validation_result["suggestions"].append("建议使用: low/medium/high")
    
    # 检查关键要点数量
    if "key_points" in data:
        if not isinstance(data["key_points"], list):
            validation_result["warnings"].append("关键要点应为数组格式")
        elif len(data["key_points"]) < 2:
            validation_result["warnings"].append("关键要点数量过少，建议至少包含2个")
    
    return validation_result

# 测试验证机制
print("🔍 测试字段验证机制：")
sample_data = {
    "title": "央行货币政策报告解读",
    "summary": "央行维持稳健货币政策，市场流动性保持合理充裕",
    "sentiment": "positive",
    "risk_level": "low",
    "key_points": ["货币政策稳健", "流动性充裕", "支持实体经济"]
}

validation = validate_financial_analysis(sample_data)
if validation["valid"]:
    print("✅ 数据验证通过")
    if validation["warnings"]:
        print("⚠️ 警告信息:")
        for warning in validation["warnings"]:
            print(f"  - {warning}")
else:
    print("❌ 数据验证失败")
    for error in validation["errors"]:
        print(f"  - {error}")

print("\\n💡 最佳实践: 在生产环境中，所有结构化输出都应经过类似验证流程")


🔍 测试字段验证机制：
✅ 数据验证通过
\n💡 最佳实践: 在生产环境中，所有结构化输出都应经过类似验证流程


## 7. 补充：置信度机制与分层复核（3 分钟）

### 什么是置信度？
**置信度（Confidence）** 是AI模型对自身预测结果可靠性的评估，取值范围通常为0-1：
- **0.9-1.0**: 高置信度 - AI非常确信结果正确
- **0.7-0.9**: 中等置信度 - AI较为确信，但存在一定不确定性  
- **0.0-0.7**: 低置信度 - AI不太确定，建议人工复核

### 银行业务中的置信度应用场景


In [9]:
# 示例：带置信度评估的财经分析
confidence_system_prompt = """你是专业的金融分析师，请在分析时同时评估自己的置信度。
置信度评估标准：
- 0.9-1.0：数据充分、逻辑清晰、结论明确
- 0.7-0.9：数据较充分、逻辑基本清晰、有一定不确定性
- 0.0-0.7：数据不足、逻辑不够清晰、结论不确定

请在JSON输出中包含confidence字段，并说明置信度评估的理由。"""

confidence_prompt = """请分析以下市场消息，重点评估分析的置信度：

市场消息：
"据传某大型银行将推出新的数字货币产品，但官方尚未确认。
市场对此反应不一，部分投资者认为这将推动金融科技发展，
另一部分投资者担心监管风险。相关银行股价出现波动。"

输出格式：
{
  "title": "新闻标题",
  "summary": "核心内容摘要", 
  "sentiment": "positive/negative/neutral",
  "confidence": 置信度数值(0-1),
  "confidence_reason": "置信度评估理由",
  "uncertainty_factors": ["不确定因素1", "不确定因素2"]
}"""

print("🎯 演示置信度评估机制：\n")


🎯 演示置信度评估机制：



In [10]:
# 执行带置信度的分析
confidence_result = call_claude_structured(
    prompt=confidence_prompt,
    system=confidence_system_prompt,
    temperature=0.2,
    prefill="{"
)

print("\n" + "="*60)
print("📊 置信度分析结果：")

try:
    conf_data = json.loads(confidence_result)
    confidence_score = conf_data.get('confidence', 0)
    
    print(f"📈 分析结果: {conf_data.get('title', 'N/A')}")
    print(f"🎯 置信度: {confidence_score:.2f}")
    print(f"💭 置信度理由: {conf_data.get('confidence_reason', 'N/A')}")
    
    # 根据置信度确定复核策略
    if confidence_score >= 0.9:
        review_strategy = "✅ 自动通过 - 高置信度"
        review_cost = "低成本"
    elif confidence_score >= 0.7:
        review_strategy = "🔍 抽样复核 - 中等置信度" 
        review_cost = "中等成本"
    else:
        review_strategy = "⚠️ 必须人工复核 - 低置信度"
        review_cost = "高成本"
    
    print(f"🛡️ 复核策略: {review_strategy}")
    print(f"💰 成本评估: {review_cost}")
    
    if 'uncertainty_factors' in conf_data:
        print("⚠️ 不确定因素:")
        for factor in conf_data['uncertainty_factors']:
            print(f"  - {factor}")
            
except json.JSONDecodeError as e:
    print(f"❌ 解析失败: {e}")



  "title": "传某大型银行将推出数字货币产品，市场反应分化",
  "summary": "市场传言某大型银行计划推出数字货币产品，但缺乏官方确认。投资者对此看法分歧，部分看好金融科技发展前景，部分担忧监管风险，相关银行股价出现波动。",
  "sentiment": "neutral",
  "confidence": 0.3,
  "confidence_reason": "置信度极低，主要原因包括：1）消息来源为传言，缺乏官方确认；2）未提供具体银行名称、产品细节等关键信息；3）市场反应描述模糊，缺乏具体数据支撑；4）股价波动程度未量化；5）整体信息过于笼统，难以进行深入分析和验证",
  "uncertainty_factors": [
    "消息真实性存疑，仅为市场传言",
    "缺乏具体银行身份和产品详情",
    "监管政策态度不明确",
    "股价波动幅度和持续性未知",
    "市场反应的量化数据缺失",
    "数字货币产品的具体形态和合规性未明"
  ]
}

📊 置信度分析结果：
📈 分析结果: 传某大型银行将推出数字货币产品，市场反应分化
🎯 置信度: 0.30
💭 置信度理由: 置信度极低，主要原因包括：1）消息来源为传言，缺乏官方确认；2）未提供具体银行名称、产品细节等关键信息；3）市场反应描述模糊，缺乏具体数据支撑；4）股价波动程度未量化；5）整体信息过于笼统，难以进行深入分析和验证
🛡️ 复核策略: ⚠️ 必须人工复核 - 低置信度
💰 成本评估: 高成本
⚠️ 不确定因素:
  - 消息真实性存疑，仅为市场传言
  - 缺乏具体银行身份和产品详情
  - 监管政策态度不明确
  - 股价波动幅度和持续性未知
  - 市场反应的量化数据缺失
  - 数字货币产品的具体形态和合规性未明


In [11]:
# 实用工具：智能复核路由系统
def intelligent_review_routing(analysis_result: Dict[str, Any]) -> Dict[str, Any]:
    """
    基于置信度和业务规则的智能复核路由
    
    参数:
    - analysis_result: AI分析结果
    
    返回:
    - 复核策略和预估成本
    """
    confidence = analysis_result.get('confidence', 0)
    risk_level = analysis_result.get('risk_level', 'high')
    sentiment = analysis_result.get('sentiment', 'neutral')
    
    # 基础置信度评估
    if confidence >= 0.9:
        base_strategy = "auto_pass"
        base_cost = 0.1  # 几乎无人工成本
    elif confidence >= 0.7:
        base_strategy = "sample_review"
        base_cost = 0.3  # 30%的人工成本
    else:
        base_strategy = "full_review"
        base_cost = 1.0  # 100%人工复核
    
    # 业务规则调整
    adjustments = []
    
    # 高风险必须人工复核
    if risk_level == 'high':
        base_strategy = "full_review"
        base_cost = 1.0
        adjustments.append("高风险等级强制人工复核")
    
    # 负面情感加强复核
    if sentiment == 'negative' and confidence < 0.8:
        if base_strategy == "auto_pass":
            base_strategy = "sample_review"
            base_cost = 0.5
        adjustments.append("负面情感降低自动化程度")
    
    # 生成最终策略
    strategy_mapping = {
        "auto_pass": "✅ 自动通过",
        "sample_review": "🔍 抽样复核（10-30%）",
        "full_review": "⚠️ 全量人工复核"
    }
    
    return {
        "strategy": strategy_mapping[base_strategy],
        "cost_ratio": base_cost,
        "estimated_time": f"{base_cost * 5:.1f}分钟/份",  # 假设单份人工复核需5分钟
        "adjustments": adjustments,
        "automation_level": f"{(1-base_cost)*100:.0f}%"
    }

# 测试智能路由
print("🤖 智能复核路由系统测试：\\n")

# 模拟不同场景的分析结果
test_scenarios = [
    {"confidence": 0.95, "risk_level": "low", "sentiment": "positive", "scenario": "央行降准消息"},
    {"confidence": 0.65, "risk_level": "medium", "sentiment": "negative", "scenario": "银行股价异常波动"},
    {"confidence": 0.45, "risk_level": "high", "sentiment": "neutral", "scenario": "监管政策不明确"}
]

for i, scenario in enumerate(test_scenarios, 1):
    routing = intelligent_review_routing(scenario)
    print(f"📋 场景{i}: {scenario['scenario']}")
    print(f"   置信度: {scenario['confidence']:.2f}")
    print(f"   复核策略: {routing['strategy']}")
    print(f"   自动化程度: {routing['automation_level']}")
    print(f"   预估时间: {routing['estimated_time']}")
    if routing['adjustments']:
        print(f"   策略调整: {'; '.join(routing['adjustments'])}")
    print()


🤖 智能复核路由系统测试：\n
📋 场景1: 央行降准消息
   置信度: 0.95
   复核策略: ✅ 自动通过
   自动化程度: 90%
   预估时间: 0.5分钟/份

📋 场景2: 银行股价异常波动
   置信度: 0.65
   复核策略: ⚠️ 全量人工复核
   自动化程度: 0%
   预估时间: 5.0分钟/份
   策略调整: 负面情感降低自动化程度

📋 场景3: 监管政策不明确
   置信度: 0.45
   复核策略: ⚠️ 全量人工复核
   自动化程度: 0%
   预估时间: 5.0分钟/份
   策略调整: 高风险等级强制人工复核

